<a href="https://colab.research.google.com/github/edgarMuno/TFM/blob/main/TFMClasificacitioEvent_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical


In [5]:
import tensorflow as tf

# 🔹 Verificar si hay GPU disponible
print("✅ Dispositivos disponibles:", tf.config.list_physical_devices())

# 🔹 Forzar uso de la GPU en TensorFlow
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # Evita uso excesivo de RAM
        tf.config.set_logical_device_configuration(
            gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=10240)]  # Asigna 10GB a la GPU
        )
        print("✅ GPU configurada correctamente.")
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ No se detectó GPU. TensorFlow usará CPU.")


✅ Dispositivos disponibles: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
⚠️ No se detectó GPU. TensorFlow usará CPU.


In [4]:
# 📂 Rutas de los archivos preprocesados
train_path = "/content/drive/MyDrive/TFM/Train/Train_process"
test_path = "/content/drive/MyDrive/TFM/Test/Test_process"
# 🔹 Hiperparámetros
SEED = 12
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 64
EPOCHS = 4
LR =  0.00003
NUM_CLASSES = 14
CLASS_LABELS = ['Abuse','Arrest','Arson','Assault','Burglary','Explosion','Fighting',"Normal",
                'RoadAccidents','Robbery','Shooting','Shoplifting','Stealing','Vandalism']


In [3]:
def npz_generator(folder_path):
    """Generador que carga los datos desde los archivos .npz"""
    class_files = sorted(os.listdir(folder_path))

    for file in class_files:
        file_path = os.path.join(folder_path, file)
        data = np.load(file_path, allow_pickle=True)
        X, y = data["X"], data["y"]

        # 🔹 Convertir imágenes a 3 canales
        X = np.repeat(X, 3, axis=-1)  # (64, 64, 1) → (64, 64, 3)

        # 🔹 Convertir etiquetas a números
        class_to_index = {class_name: idx for idx, class_name in enumerate(CLASS_LABELS)}
        y = np.array([class_to_index[label] for label in y])  # Convertir etiquetas a índice

        # 🔹 One-hot encoding
        y = tf.keras.utils.to_categorical(y, num_classes=NUM_CLASSES)

        for i in range(len(X)):  # Iterar sobre cada imagen
            yield X[i], y[i]  # Retornar una imagen a la vez


def npz_generator(folder_path):
    """Generador que carga los datos desde los archivos .npz en lotes pequeños"""
    class_files = sorted(os.listdir(folder_path))  # Listar los archivos en la carpeta

    for file in class_files:
        file_path = os.path.join(folder_path, file)
        data = np.load(file_path, allow_pickle=True)
        X, y = data["X"], data["y"]

        # 🔹 Convertir imágenes de 1 canal a 3 canales (RGB para DenseNet121)
        X = np.repeat(X, 3, axis=-1)  # (64, 64, 1) → (64, 64, 3)

        # 🔹 Convertir etiquetas a números
        class_to_index = {class_name: idx for idx, class_name in enumerate(CLASS_LABELS)}
        y = np.array([class_to_index[label] for label in y])  # Convertir etiquetas

        # 🔹 One-hot encoding
        y = to_categorical(y, num_classes=NUM_CLASSES)

        for i in range(0, len(X), BATCH_SIZE):
            yield X[i:i+BATCH_SIZE], y[i:i+BATCH_SIZE]


MEzclamos los datos

# 🔹 Crear datasets en streaming con mezcla de datos (shuffle)
train_dataset = tf.data.Dataset.from_generator(
    lambda: npz_generator(train_path),
    output_signature=(
        tf.TensorSpec(shape=(None, IMG_HEIGHT, IMG_WIDTH, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, NUM_CLASSES), dtype=tf.float32)
    )
).shuffle(buffer_size=BATCH_SIZE * 10).prefetch(tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_generator(
    lambda: npz_generator(test_path),
    output_signature=(
        tf.TensorSpec(shape=(None, IMG_HEIGHT, IMG_WIDTH, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, NUM_CLASSES), dtype=tf.float32)
    )
).prefetch(tf.data.experimental.AUTOTUNE)

print("✅ Datasets en streaming cargados correctamente con mezcla de datos.")


El mismo de arriab pero evita desborde de ram

In [6]:
train_dataset = tf.data.Dataset.from_generator(
    lambda: npz_generator(train_path),
    output_signature=(
        tf.TensorSpec(shape=(IMG_HEIGHT, IMG_WIDTH, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(NUM_CLASSES,), dtype=tf.float32)
    )
).shuffle(buffer_size=BATCH_SIZE * 2).batch(BATCH_SIZE).cache(filename="/content/train_cache").prefetch(tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_generator(
    lambda: npz_generator(test_path),
    output_signature=(
        tf.TensorSpec(shape=(IMG_HEIGHT, IMG_WIDTH, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(NUM_CLASSES,), dtype=tf.float32)
    )
).shuffle(buffer_size=BATCH_SIZE * 2).batch(BATCH_SIZE).cache(filename="/content/test_cache").prefetch(tf.data.experimental.AUTOTUNE)


In [8]:
# 📂 Contar el total de imágenes en entrenamiento y validación
total_train_images = sum([np.load(os.path.join(train_path, file))["X"].shape[0] for file in sorted(os.listdir(train_path))])
total_test_images = sum([np.load(os.path.join(test_path, file))["X"].shape[0] for file in sorted(os.listdir(test_path))])

# 🔹 Calcular los steps por epoch
steps_per_epoch = total_train_images // BATCH_SIZE
validation_steps = total_test_images // BATCH_SIZE

print(f"✅ Steps per epoch: {steps_per_epoch}, Validation steps: {validation_steps}")


✅ Steps per epoch: 16909, Validation steps: 1739


In [9]:
# 🔹 Definir el porcentaje de datos a usar (ejemplo: 10% del total)
train_fraction = 0.1  # Usar solo el 10% del dataset
test_fraction = 0.1   # Usar solo el 10% del dataset

# 🔹 Calcular la cantidad de pasos a tomar
subset_train_steps = int(steps_per_epoch * train_fraction)
subset_test_steps = int(validation_steps * test_fraction)

# 🔹 Crear datasets reducidos con solo un porcentaje de los datos
train_subset = train_dataset.take(subset_train_steps)
test_subset = test_dataset.take(subset_test_steps)

print(f"✅ Se entrenará con el {train_fraction*100:.0f}% del dataset: {subset_train_steps} steps en Train y {subset_test_steps} en Test")


✅ Se entrenará con el 10% del dataset: 1690 steps en Train y 173 en Test


In [10]:
def feature_extractor(inputs):
    base_model = tf.keras.applications.DenseNet121(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                                                   include_top=False,
                                                   weights="imagenet")(inputs)
    return base_model

def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax", name="classification")(x)

    return x

def final_model(inputs):
    densenet_feature_extractor = feature_extractor(inputs)
    classification_output = classifier(densenet_feature_extractor)

    return classification_output

def define_compile_model():
    inputs = tf.keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    classification_output = final_model(inputs)
    model = tf.keras.Model(inputs=inputs, outputs=classification_output)

    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=LR),
                loss='categorical_crossentropy',
                metrics=[tf.keras.metrics.AUC()])

    return model

# 🔹 Crear el modelo
model = define_compile_model()
model.summary()


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ densenet121 (Functional)             │ (None, 2, 2, 1024)          │       7,037,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1024)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         262,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1024)                │         263,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ classification (Dense)               │ (None, 14)                  │           7,182 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,095,054 (30.88 MB)

 Trainable params: 8,011,406 (30.56 MB)

 Non-trainable params: 83,648 (326.75 KB)

In [11]:
# 🔹 Entrenar con solo un porcentaje del dataset
history = model.fit(
    train_subset,
    validation_data=test_subset,
    epochs=EPOCHS,
    steps_per_epoch=subset_train_steps,
    validation_steps=subset_test_steps
)


Epoch 1/4
 118/1690 ━━━━━━━━━━━━━━━━━━━━ 1:59:40 5s/step - auc: 0.6496 - loss: 2.6326

KeyboardInterrupt: 

In [11]:
# 🔹 Entrenar el modelo con validación en cada epoch
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)


Epoch 1/4
   22/16909 ━━━━━━━━━━━━━━━━━━━━ 18:16 65ms/step - auc: 0.5060 - loss: 3.1440

KeyboardInterrupt: 